<a href="https://colab.research.google.com/github/Jaydip1989/AnimalClassifier/blob/master/SwitchOnNewAssignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
image_dir="/content/drive/My Drive/SwitchOnNew/images/"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential, load_model, model_from_json
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import Adam

In [ ]:
num_classes = 2
img_rows,img_cols = 64,64
batch_size = 100

In [ ]:
datagen = ImageDataGenerator(rescale = 1./255,
                             validation_split=0.2)

In [ ]:
train_generator = datagen.flow_from_directory(directory=image_dir,
                                              target_size=(img_rows, img_cols),
                                              batch_size = batch_size,
                                              shuffle = True,
                                              class_mode = "categorical",
                                              subset="training")

In [ ]:
val_generator = datagen.flow_from_directory(directory = image_dir,
                                            target_size=(img_rows, img_cols),
                                            batch_size = batch_size,
                                            shuffle = True,
                                            subset="validation",
                                            class_mode = "categorical")

In [ ]:
train_samples = train_generator.n//train_generator.batch_size
val_samples = val_generator.n//val_generator.batch_size


In [ ]:
from keras.applications import VGG16
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers import Flatten, Dense
from keras.models import Sequential, Model

In [ ]:
img_rows, img_cols = 224, 224
vgg = VGG16(weights="imagenet",
            input_shape=(224,224, 3),
            include_top = False)
for layer in vgg.layers:
  layer.trainable = False

def top_model(bottom_model, num_classes):
  top_model = bottom_model.output
  top_model = GlobalAveragePooling2D()(top_model)
  top_model = Dense(1024, activation="relu")(top_model)
  top_model = Dropout(0.25)(top_model)
  top_model = Dense(1024, activation="relu")(top_model)
  top_model = Dropout(0.25)(top_model)
  top_model = Dense(num_classes, activation="softmax")(top_model)
  return top_model

FC_Head = top_model(vgg, num_classes)
vggmodel = Model(inputs = vgg.input, outputs = FC_Head)
vggmodel.summary()

In [ ]:
vggmodel.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

In [ ]:
epochs = 5
vggmodel.fit(train_generator,
          epochs=epochs,
          steps_per_epoch = train_samples,
          validation_data = val_generator,
          validation_steps = val_samples)

In [ ]:
scores = vggmodel.evaluate(val_generator, steps = val_samples, verbose=1)
print("\n Validation Accuracy:%.3f Validation Loss:%.3f"%(scores[1]*100,scores[0]))

In [ ]:
vggmodel.save('VGG16Netmodel_badimages.h5')
vgg_classifier = load_model('VGG16Netmodel_badimages.h5')

In [ ]:
vggmodel.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy'])

In [ ]:
epochs = 15
vggmodel.fit(train_generator,
          epochs=epochs,
          steps_per_epoch = train_samples,
          validation_data = val_generator,
          validation_steps = val_samples)

In [ ]:
scores = vggmodel.evaluate(val_generator, steps = val_samples, verbose=1)
print("\n Validation Accuracy:%.3f Validation Loss:%.3f"%(scores[1]*100,scores[0]))

In [ ]:
vggmodel.save('VGG16Netmodel2_badimages.h5')
vgg_classifier2 = load_model('VGG16Netmodel2_badimages.h5')

In [ ]:
prediction = vgg_classifier.predict(val_generator, steps=val_samples,verbose=1)
pred_labels = np.argmax(prediction, axis=1)
print(pred_labels)

In [ ]:
from PIL import Image 
IMG = Image.open('/content/drive/My Drive/SwitchOnNew/good_or_bad_image.png')
IMG = IMG.resize((224,224))
IMG = np.array(IMG)
IMG = np.true_divide(IMG,255)
IMG = IMG.reshape(1, 224,224, 3)

### The predictions below are the predictions from the model with learning rate of 0.001

In [ ]:
predictions = vgg_classifier.predict(IMG)
predicted_classes = np.argmax(vgg_classifier.predict(IMG),axis=1)
print(predictions, predicted_classes)

In [ ]:
classes = {
    'TRAIN':['bad_images', 'good_images'],
    'VALIDATION':['bad_images','good_images']}

predicted_class = classes['VALIDATION'][predicted_classes[0]]
print("I think this image is among the {}.".format(predicted_class.lower()))

In [ ]:
predictions = vgg_classifier2.predict(val_generator, steps=val_samples,verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)

In [ ]:
preds = vgg_classifier2.predict(val_generator, steps=val_samples,verbose=1)
pred_labels = np.argmax(preds, axis=1)
print(pred_labels)

In [ ]:
preds = vgg_classifier2.predict(IMG)
predicted_classes = np.argmax(vgg_classifier2.predict(IMG),axis=1)
print(preds, predicted_classes)

In [ ]:
classes = {
    'TRAIN':['bad_images', 'good_images'],
    'VALIDATION':['bad_images','good_images']}

predicted_class = classes['VALIDATION'][predicted_classes[0]]
print("I think this image is among the {}.".format(predicted_class.lower()))

In [ ]:
from PIL import Image 
image = Image.open('/content/drive/My Drive/SwitchOnNew/good_or_bad_image2.png')
image = image.resize((224,224))
image = np.array(image)
image = np.true_divide(image,255)
image = image.reshape(1, 224,224, 3)

In [ ]:
preds = vgg_classifier2.predict(image)
predicted_classes = np.argmax(vgg_classifier2.predict(image),axis=1)
print(preds, predicted_classes)

In [ ]:
classes = {
    'TRAIN':['bad_images', 'good_images'],
    'VALIDATION':['bad_images','good_images']}

predicted_class = classes['VALIDATION'][predicted_classes[0]]
print("I think this image is among the {}.".format(predicted_class.lower()))